In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.bayesflow import stochastic_tensor, stochastic_graph
from tensorflow.contrib.distributions import Normal

In [2]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [3]:
mnist = read_data_sets('/home/hadoop/data/mnist', one_hot=True)

Extracting /home/hadoop/data/mnist/train-images-idx3-ubyte.gz
Extracting /home/hadoop/data/mnist/train-labels-idx1-ubyte.gz
Extracting /home/hadoop/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/hadoop/data/mnist/t10k-labels-idx1-ubyte.gz


In [4]:
class DenoisedAutoEncoder:
    def __init__(self, input_size, hidden_size, batch_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        x = tf.placeholder(tf.float32, [batch_size, input_size])
        # stochastic 
        st = stochastic_graph.stochastic_tensor.StochasticTensor(
            Normal, 
            mu=tf.zeros([batch_size, input_size]), 
            sigma=tf.constant(0.01 * np.ones([batch_size, input_size]), dtype=tf.float32)
        )
        x_noised = x + tf.convert_to_tensor(st, dtype=tf.float32)
        
        w = tf.Variable(tf.truncated_normal([input_size, hidden_size], 0, 0.1))
        hb = tf.Variable(tf.zeros([hidden_size]))
        y = tf.nn.sigmoid(tf.matmul(x_noised, w) + hb)
        
        vb = tf.Variable(tf.zeros([input_size]))
        z = tf.nn.sigmoid(tf.matmul(y, tf.transpose(w)) + vb)
        
        self.input = x
        self.z = z
        self.loss = -tf.reduce_mean(x * tf.log(z) + (1-x) * tf.log(1-z))
        self.train_op = tf.train.AdamOptimizer(0.005).minimize(self.loss)    

In [5]:
batch_size = 512

with tf.Session() as sess:
    model = DenoisedAutoEncoder(784, 100, batch_size)
    sess.run(tf.initialize_all_variables())
    for i in range(1, 1001):
        _x, _y = mnist.train.next_batch(batch_size)
        _loss, _ = sess.run([model.loss, model.train_op], feed_dict={
                model.input : _x
            })
        if i % 100 == 0:
            print('batch ', i, _loss)

batch  100 0.173669
batch  200 0.143797
batch  300 0.130764
batch  400 0.124926
batch  500 0.119534
batch  600 0.116786
batch  700 0.113251
batch  800 0.108063
batch  900 0.108938
batch  1000 0.107392
